# Animating Subset Construction (NFA to DFA)

This notebook generates and displays the animation of the subset conversion from an NFA to DFA. After executing each cell, you will be able to view the animation at the end of the notebook.

Import `gvanim`, which was installed using `pip install GraphvizAnim`. This module is used to generate the interactive animations. 

In [8]:
from gvanim import Animation
from gvanim.jupyter import interactive
ga_new = Animation() # variable used to represent the animation


In the section below, we introduce all the classes and methods needed to produce a Nondeterministic Finite State Machine. These methods were referenced from the lecture notes (02 Regular Languages).

In [9]:
class FiniteStateMachine:
    def __init__(self, T, Q, R, q0, F):
        self.T, self.Q, self.R, self.q0, self.F = T, Q, R, q0, F
    def __repr__(self):
        return str(self.q0) + '\n' + ' '.join(self.F) + '\n' + \
               '\n'.join(r[0] + ' ' + r[1] + ' → ' + r[2] for r in self.R)

def parseFSM(fsm: str) -> FiniteStateMachine:
    fsm = [line for line in fsm.split('\n') if line.strip() != '']
    q0 = fsm[0].split()[0] # first line: initialstate
    F = set(fsm[1].split()) # second line: finalstate, finalstate, ...
    R = set()
    for line in fsm[2:]: # all subsequent lines: "source symbol → target"
        l, r = line.split('→')
        R |= {(l.split()[0], l.split()[1], r.split()[0])}
    T = {r[1] for r in R}
    Q = {q0} | F | {r[0] for r in R} | {r[2] for r in R}
    return FiniteStateMachine(T, Q, R, q0, F)

class Choice:
    def __init__(self, e1, e2): self.e1, self.e2 = e1, e2
class Conc:
    def __init__(self, e1, e2): self.e1, self.e2 = e1, e2
class Star:
    def __init__(self, e): self.e = e
        
def syntaxgraph(re):
    global node, T
    if re == '': return {(None, None)}
    elif type(re) == str:
        node += 1; T.add(re); return {(None, (re, str(node))), ((re, str(node)), None)}
    elif type(re) == Choice:
        return syntaxgraph(re.e1) | syntaxgraph(re.e2)
    elif type(re) == Conc:
        g1, g2 = syntaxgraph(re.e1), syntaxgraph(re.e2)
        return {(a, b) for (a, b) in g1 if b} | \
               {(a, b) for (a, b) in g2 if a} | \
               {(a, b) for (a, c) in g1 for (d, b) in g2 if not c and not d}
    elif type(re) == Star:
        g = syntaxgraph(re.e)
        return {(None, None)} | g | \
               {(a, b) for (a, c) in g for (d, b) in g if not c and not d}
    else: raise Exception('not a regular expression')
        
def convertRegExToFSM(re):
    global node, T; node, T = 0, set()
    g = syntaxgraph(re)
    Q = {str(n) for n in range(node + 1)}
    R = {('0', b[0], b[1]) for (a, b) in g if not a and b} | \
        {(a[1], b[0], b[1]) for (a, b) in g if a and b}
    F = {a[1] for (a, b) in g if a and not b} | ({'0'} if (None, None) in g else set())
    output = FiniteStateMachine(T, Q, R, '0', F)
    output = str(output)
    return output

###### Defining the Nondeterministic FSM

First, we need to define the NFA using the methods defined above. We do this by defining a regular expression, and convert that regex into an FSM using `convertRegExToFSM()`. 

In [10]:
E4 = Choice(Conc(Conc('a', Star('a')), 'b'), Conc(Conc('a', Star('a')), 'c'))
A4 = convertRegExToFSM(E4); A4 = A4.splitlines()
for i in A4:
    print(i)

0
3 6
4 a → 5
1 b → 3
4 c → 6
2 a → 2
2 b → 3
5 a → 5
1 a → 2
5 c → 6
0 a → 4
0 a → 1


In the section below, we introduce the additional classes and methods needed to produce a Deterministic Finite State Machine. These methods were referenced from the lecture notes (02 Regular Languages).

In [11]:
def convertRegExToFSM(re):
    global node, T; node, T = 0, set()
    g = syntaxgraph(re)
    Q = {str(n) for n in range(node + 1)}
    R = {('0', b[0], b[1]) for (a, b) in g if not a and b} | \
        {(a[1], b[0], b[1]) for (a, b) in g if a and b}
    F = {a[1] for (a, b) in g if a and not b} | ({'0'} if (None, None) in g else set())
    return FiniteStateMachine(T, Q, R, '0', F)

def string(s: set) -> str:
    return '{' + ', '.join(e for e in s) + '}'

def deterministicFSM(fsm: FiniteStateMachine) -> FiniteStateMachine:
    qq0 = string({fsm.q0})
    QQ, RR, visited = {qq0}, set(), set()
    #print(QQ, RR, visited)
    while visited != QQ:
        qq = (QQ - visited).pop(); visited |= {qq}
        for t in fsm.T:
            rr = {r for (q, u, r) in fsm.R if u == t and q in qq}
            if rr != set(): QQ |= {string(rr)}; RR |= {(qq, t, string(rr))}
        #print(QQ, RR, visited)
    FF = {qq for qq in QQ for f in fsm.F if f in qq}
    output = FiniteStateMachine(fsm.T, QQ, RR, qq0, FF)
    output = str(output)
    return output

###### Defining the Deterministic FSM

Next, we need to define the equivalent DFA for the above NFA. This is done by inputting the NFA into the method `deterministicFSM()`. We then split the DFA into a list of strings. 

In [12]:
print("Nondeterministic FSM:")
for i in A4:
    print(i)
A4NFA = convertRegExToFSM(E4)
A4det = deterministicFSM(A4NFA); A4det = A4det.splitlines()
print("Deterministic FSM:")
for i in range(len(A4det)): 
    A4det[i] = A4det[i]
    print(A4det[i])

Nondeterministic FSM:
0
3 6
4 a → 5
1 b → 3
4 c → 6
2 a → 2
2 b → 3
5 a → 5
1 a → 2
5 c → 6
0 a → 4
0 a → 1
Deterministic FSM:
{0}
{6} {3}
{5, 2} a → {5, 2}
{1, 4} c → {6}
{5, 2} b → {3}
{1, 4} b → {3}
{5, 2} c → {6}
{1, 4} a → {5, 2}
{0} a → {1, 4}


### Animating the Conversion from NFA to DFA:

In this stage, we define a method for animating the subset conversion from NFA to DFA.<br> 
The method `convertNFA(NFA, DFA)` takes two input values, an NFA and a DFA. Each input value is a list of lists. The method iterates through both lists, converting each element in the NFA list to the appropriate element in the DFA list. While converting each element, the method makes the corresponding changes to the animation. 

In [13]:
def convertNFA(NFA,DFA):
    eventNFA = [] # order of events for NFA
    eventDFA = [] # order of events for DFA
    replaced = []
    new_stateNFA = [] # list of new states in NFA list 
    new_stateDFA = [] # list of new states in DFA list 
    # assign each state and transition in NFA 
    
    for i in range(2,len(NFA)):
        oldNFA = NFA[i][0:1] # old state 
        transitionNFA = NFA[i][2:3] # transition 
        newNFA = NFA[i][NFA[i].find("→")+2:] # new state
        # prints model of NFA
        ga_new.add_edge(oldNFA,newNFA) 
        ga_new.label_edge(oldNFA,newNFA,transitionNFA)
        eventNFA.append([oldNFA,transitionNFA,newNFA]) # states and transitions in NFA
        new_stateNFA.append(newNFA)
    ga_new.next_step()
    
    for i in range(2,len(DFA)):
        oldDFA = DFA[i][DFA[i].find("{")+1:DFA[i].find("}")]
        end_bracket = (DFA[i].find("}")) # index of first end bracket 
        transitionDFA = DFA[i][end_bracket+2]
        newDFA = DFA[i][end_bracket+6:len(DFA[i])].replace('{','').replace('}','')
        eventDFA.append([oldDFA,transitionDFA, newDFA]) # states and transitions in DFA
        new_stateDFA.append(newDFA)
        
    eventDFA = sorted(eventDFA)
    eventNFA = sorted(eventNFA)
    
    # iterate through both lists, converting NFA -> DFA
    while (eventNFA != eventDFA): # end when both lists are equal 
        for i in eventNFA:
            for j in eventDFA:
                if i[0:2] == j[0:2]: # if oldNFA in DFA 
                    index = (eventNFA.index(i)) # index of value to be replaced with combined state 
                    replaced = i 
                    ga_new.remove_edge(i[0],i[2]) # remove old edge 
                    eventNFA[index] = j # replace newNFA with newDFA
                    ga_new.add_edge(i[0],j[2]) # add new edge 
                    ga_new.label_edge(i[0],j[2],i[1])
                    ga_new.next_step()
                if replaced != []: # change starting state in rest of NFA list 
                    for x in eventNFA:
                        if x[0] == replaced[2]:
                            x[0] = j[2]
                replaced = []
        eventNFA = set(map(tuple, eventNFA)) # remove duplicates 
        eventNFA = sorted(list(map(list, eventNFA)))
        eventDFA = sorted(eventDFA)

    # remove unnecessary nodes 
    new_stateNFA = list(set(new_stateNFA))
    for i in new_stateNFA:
        if i not in new_stateDFA:
            ga_new.remove_node(i)
            ga_new.next_step()
    return eventNFA

convertNFA(A4,A4det)

[['0', 'a', '1, 4'],
 ['1, 4', 'a', '5, 2'],
 ['1, 4', 'b', '3'],
 ['1, 4', 'c', '6'],
 ['5, 2', 'a', '5, 2'],
 ['5, 2', 'b', '3'],
 ['5, 2', 'c', '6']]

### Running the Animation of the Conversion from NFA to DFA:

Here, we call the method `interactive()`, which was imported with the module `gvanim`. This method generates the interactive animation using the previously defined methods, `convertNFA()`.

Move the slider from left to right to view the animation of the subset conversion. 

In [14]:
interactive(ga_new,600)

interactive(children=(IntSlider(value=0, description='n', max=15), Output()), _dom_classes=('widget-interact',…